# Puertas cuánticas personalizadas

In [57]:
import numpy as np
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.quilbase import DefGate, DefPermutationGate, Declare
from pyquil.quilatom import Parameter, quil_cos, quil_sin
import pyquil.quilatom as qa

In [58]:
qvm = get_qc('9q-square-qvm')

## Puertas no paramétricas

### SQRT-X

In [59]:
sqrt_x = np.array([
    [0.5 + 0.5j, 0.5 - 0.5j],
    [0.5 - 0.5j, 0.5 + 0.5j]
])

sqrt_x_definition = DefGate("SQRT_X", sqrt_x)
SQRT_X = sqrt_x_definition.get_constructor()

In [ ]:
prog_sqrt_x = Program(
    Declare("ro", "BIT", 1),
    sqrt_x_definition,
    SQRT_X(0),
    MEASURE(0, ("ro", 0))
)

# Número alto para que la prob se acerque más al 50%
n_shots = 1000
results = []
for _ in range(n_shots):
    result = qvm.run(qvm.compile(prog_sqrt_x)).get_register_map().get("ro")
    results.append(result[0, 0])

result_sqrt_x = np.array(results)

print(f"\nPromedio: {np.mean(result_sqrt_x):.3f}")
print("(Valor esperado: ~0.5, ya que SQRT-X crea una superposición)")
print(f"Conteo de 0s: {np.sum(result_sqrt_x == 0)}")
print(f"Conteo de 1s: {np.sum(result_sqrt_x == 1)}")


Promedio: 0.491
(Valor esperado: ~0.5, ya que SQRT-X crea una superposición)
Conteo de 0s: 509
Conteo de 1s: 491


In [61]:
prog_sqrt_x_verify = Program(
    Declare("ro", "BIT", 1),
    sqrt_x_definition,
    SQRT_X(0),
    SQRT_X(0),
    MEASURE(0, ("ro", 0))
)

result_verify = qvm.run(qvm.compile(prog_sqrt_x_verify)).get_register_map().get("ro")
print("\nResultados de medición:")
print(result_verify)
print(f"\nPromedio: {np.mean(result_verify):.2f}")
print("(Valor esperado: ~1.0, equivalente a aplicar X)")


Resultados de medición:
[[1]]

Promedio: 1.00
(Valor esperado: ~1.0, equivalente a aplicar X)


### SQRT-Y

In [62]:
sqrt_y = np.array([
    [0.5 + 0.5j, -0.5 - 0.5j],
    [0.5 + 0.5j,  0.5 + 0.5j]
])

sqrt_y_definition = DefGate("SQRT_Y", sqrt_y)
SQRT_Y = sqrt_y_definition.get_constructor()

In [63]:
prog_sqrt_y = Program(
    Declare("ro", "BIT", 1),
    sqrt_y_definition,
    SQRT_Y(0),
    MEASURE(0, ("ro", 0))
)

result_sqrt_y = qvm.run(qvm.compile(prog_sqrt_y)).get_register_map().get("ro")
print("\nResultados de medición:")
print(result_sqrt_y)
print(f"\nPromedio: {np.mean(result_sqrt_y):.2f}")


Resultados de medición:
[[0]]

Promedio: 0.00


In [64]:
prog_sqrt_y_verify = Program(
    Declare("ro", "BIT", 1),
    sqrt_y_definition,
    SQRT_Y(0),
    SQRT_Y(0),
    MEASURE(0, ("ro", 0))
)

result_verify_y = qvm.run(qvm.compile(prog_sqrt_y_verify)).get_register_map().get("ro")
print(f"Promedio: {np.mean(result_verify_y):.2f}")
print("(Valor esperado: ~1.0, equivalente a aplicar Y)")

Promedio: 1.00
(Valor esperado: ~1.0, equivalente a aplicar Y)


## Puertas paramétricas

### CRX

In [65]:
theta = Parameter('theta')

crx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta / 2), -1j * quil_sin(theta / 2)],
    [0, 0, -1j * quil_sin(theta / 2), quil_cos(theta / 2)]
])

crx_definition = DefGate("CRX_CUSTOM", crx, [theta])
CRX_CUSTOM = crx_definition.get_constructor()

In [66]:
prog_crx = Program(
    Declare("ro", "BIT", 2),
    crx_definition,
    H(0),
    CRX_CUSTOM(np.pi/2)(0, 1),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1))
)

result_crx = qvm.run(qvm.compile(prog_crx)).get_register_map().get("ro")
print("\nResultados de medición:")
print(result_crx)
print(f"\nMedia q0: {np.mean(result_crx[:, 0]):.2f}")
print(f"Media q1: {np.mean(result_crx[:, 1]):.2f}")


Resultados de medición:
[[0 0]]

Media q0: 0.00
Media q1: 0.00


In [67]:
prog_crx_control1 = Program(
    Declare("ro", "BIT", 2),
    crx_definition,
    X(0),  # Poner control en |1⟩
    CRX_CUSTOM(np.pi/2)(0, 1),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1))
)

result_crx_1 = qvm.run(qvm.compile(prog_crx_control1)).get_register_map().get("ro")
print(f"Promedio q0: {np.mean(result_crx_1[:, 0]):.2f}")
print(f"Promedio q1: {np.mean(result_crx_1[:, 1]):.2f}")
print("(q1 debería estar en superposición)")

Promedio q0: 1.00
Promedio q1: 1.00
(q1 debería estar en superposición)


### CRY

In [68]:
theta_y = Parameter('theta_y')

cry = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, quil_cos(theta_y / 2), -quil_sin(theta_y / 2)],
    [0, 0, quil_sin(theta_y / 2), quil_cos(theta_y / 2)]
])

cry_definition = DefGate("CRY_CUSTOM", cry, [theta_y])
CRY_CUSTOM = cry_definition.get_constructor()

In [69]:
prog_cry = Program(
    Declare("ro", "BIT", 2),
    cry_definition,
    H(0),
    CRY_CUSTOM(np.pi/3)(0, 1),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1))
)

result_cry = qvm.run(qvm.compile(prog_cry)).get_register_map().get("ro")
print("\nResultados de medición:")
print(result_cry)
print(f"\nPromedio q0: {np.mean(result_cry[:, 0]):.2f}")
print(f"Promedio q1: {np.mean(result_cry[:, 1]):.2f}")


Resultados de medición:
[[1 0]]

Promedio q0: 1.00
Promedio q1: 0.00


### Puerta U1

In [70]:
theta_u1 = Parameter('theta_u1')

u1 = np.array([
    [1, 0],
    [0, qa.quil_exp(1j * theta_u1)]
])

u1_definition = DefGate("U1_CUSTOM", u1, [theta_u1])
U1_CUSTOM = u1_definition.get_constructor()

In [ ]:
prog_u1 = Program(
    Declare("ro", "BIT", 1),
    u1_definition,
    H(0),
    U1_CUSTOM(np.pi/4)(0),
    MEASURE(0, ("ro", 0))
)

result_u1 = qvm.run(qvm.compile(prog_u1)).get_register_map().get("ro")
print("\nResultados de medición:")
print(result_u1)
print(f"\nPromedio: {np.mean(result_u1):.2f}")
print("(U1 solo añade fase, no cambia probabilidades de medición)")


Resultados de medición:
[[0]]

Promedio: 0.000
(U1 solo añade fase, no cambia probabilidades de medición)


## Puertas permutadas

### CCX

In [72]:
ccnot_gate = DefPermutationGate(
    "PERMUTATION_CCNOT", 
    [
        0, 1, 2, 3, 4, 5, 7, 6
    ]
)
CCNOT_G = ccnot_gate.get_constructor()

In [73]:
prog_ccnot_00 = Program(
    Declare("ro", "BIT", 3),
    ccnot_gate,
    CCNOT_G(0, 1, 2),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2))
)

result_ccnot_00 = qvm.run(qvm.compile(prog_ccnot_00)).get_register_map().get("ro")
print("\nResultados:")
print(result_ccnot_00)
print(f"\nEstado promedio: |{np.mean(result_ccnot_00[:, 0]):.0f}{np.mean(result_ccnot_00[:, 1]):.0f}{np.mean(result_ccnot_00[:, 2]):.0f}⟩")
print("(Esperado: |000⟩ - no hay cambio)")


Resultados:
[[0 0 0]]

Estado promedio: |000⟩
(Esperado: |000⟩ - no hay cambio)


In [74]:
prog_ccnot_11 = Program(
    Declare("ro", "BIT", 3),
    ccnot_gate,
    X(0),
    X(1),
    CCNOT_G(0, 1, 2),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2))
)

result_ccnot_11 = qvm.run(qvm.compile(prog_ccnot_11)).get_register_map().get("ro")
print("\nResultados:")
print(result_ccnot_11)
print(f"\nEstado promedio: |{np.mean(result_ccnot_11[:, 0]):.0f}{np.mean(result_ccnot_11[:, 1]):.0f}{np.mean(result_ccnot_11[:, 2]):.0f}⟩")
print("(Esperado: |111⟩ - el target se voltea)")


Resultados:
[[1 1 1]]

Estado promedio: |111⟩
(Esperado: |111⟩ - el target se voltea)


In [75]:
prog_ccnot_10 = Program(
    Declare("ro", "BIT", 3),
    ccnot_gate,
    X(0),  # Solo q0 en |1⟩
    CCNOT_G(0, 1, 2),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2))
)

result_ccnot_10 = qvm.run(qvm.compile(prog_ccnot_10)).get_register_map().get("ro")
print(f"Estado promedio: |{np.mean(result_ccnot_10[:, 0]):.0f}{np.mean(result_ccnot_10[:, 1]):.0f}{np.mean(result_ccnot_10[:, 2]):.0f}⟩")
print("(Esperado: |100⟩ - no hay cambio, necesita ambos controles en |1⟩)")

Estado promedio: |100⟩
(Esperado: |100⟩ - no hay cambio, necesita ambos controles en |1⟩)


### CCCX

In [76]:
cccx_gate = DefPermutationGate(
    "PERMUTATION_CCCX", 
    [
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 14
    ]
)
CCCX_G = cccx_gate.get_constructor()

In [77]:
prog_cccx_111 = Program(
    Declare("ro", "BIT", 4),
    cccx_gate,
    X(0),
    X(1),
    X(2),
    CCCX_G(0, 1, 2, 3),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2)),
    MEASURE(3, ("ro", 3))
)

result_cccx_111 = qvm.run(qvm.compile(prog_cccx_111)).get_register_map().get("ro")
print("\nResultados:")
print(result_cccx_111)
print(f"\nEstado promedio: |{np.mean(result_cccx_111[:, 0]):.0f}{np.mean(result_cccx_111[:, 1]):.0f}{np.mean(result_cccx_111[:, 2]):.0f}{np.mean(result_cccx_111[:, 3]):.0f}⟩")
print("(Esperado: |1111⟩ - el target se voltea)")


Resultados:
[[1 1 1 1]]

Estado promedio: |1111⟩
(Esperado: |1111⟩ - el target se voltea)


In [78]:
prog_cccx_110 = Program(
    Declare("ro", "BIT", 4),
    cccx_gate,
    X(0),
    X(1),
    CCCX_G(0, 1, 2, 3),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2)),
    MEASURE(3, ("ro", 3))
)

print("\nPrograma CCCX con controles en |110⟩:")
result_cccx_110 = qvm.run(qvm.compile(prog_cccx_110)).get_register_map().get("ro")
print(f"Estado promedio: |{np.mean(result_cccx_110[:, 0]):.0f}{np.mean(result_cccx_110[:, 1]):.0f}{np.mean(result_cccx_110[:, 2]):.0f}{np.mean(result_cccx_110[:, 3]):.0f}⟩")
print("(Esperado: |1100⟩ - no hay cambio, necesita los 3 controles en |1⟩)")


Programa CCCX con controles en |110⟩:
Estado promedio: |1100⟩
(Esperado: |1100⟩ - no hay cambio, necesita los 3 controles en |1⟩)
